In [ ]:
words = open("names.txt").read().splitlines()

In [ ]:
words[:5]

In [ ]:
len(words)

In [ ]:
min(len(w) for w in words), max(len(w) for w in words)

In [ ]:
b = {}
for w in words: 
    chs = ["<S>"] + list(w) + ["<E>"]
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) + 1

In [ ]:
sorted(b.items(), key=lambda kv: -kv[1])[:10]

In [ ]:
import torch

In [ ]:
N = torch.zeros((27, 27), dtype=torch.int32)

In [ ]:
chars = sorted(list(set("".join(words))))
stoi = {s:i + 1 for i, s in enumerate(chars)}
stoi["."] = 0 
itos = {i:s for s, i in stoi.items()}

In [ ]:
for w in words:
    chs = ["."] + list(w) + ["."]
    for ch1, ch2 in zip(chs, chs[1:]):
        N[stoi[ch1], stoi[ch2]] += 1

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(15, 15))
plt.imshow(N, cmap="Blues")
for y in range(27):
    for x in range(27):
        plt.text(x, y, itos[y] + itos[x], ha="center", va="bottom", color="gray")
        plt.text(x, y, N[y, x].item(), ha="center", va="top", color="gray")
plt.axis("off");

In [ ]:
N[0]

In [ ]:
p = N[0].float()
p /= p.sum()
p

In [ ]:
p.shape

In [ ]:
g = torch.Generator().manual_seed(2**31 - 1)
idx = torch.multinomial(p, 1, True, generator=g).item()
itos[idx]

In [ ]:
P = (N + 1).float() 
P /= P.sum(1, True)

In [ ]:
for _ in range(5):
    idx, out = 0, []
    while True:
        p = P[idx]
        idx = torch.multinomial(p, 1, True, generator=g).item()
        out.append(itos[idx])
        if idx == stoi["."]:
            break
    print("".join(out))